<a href="https://colab.research.google.com/github/HimalayanSaswataBose/POS_Tagging/blob/main/HMM_POS_Tagger.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import random
import numpy as np
import pandas as pd
import pprint, time

Read data

In [ ]:
test_tagged_words = []
with open("test.txt", mode = "r") as f:
  g = f.read()
  k = g.split('\n')
  for i in k:
    t = i.split('\t')
    if t[0] != '':
      test_tagged_words.append(t)
print(test_tagged_words)
print(len(test_tagged_words))


[['সত্যব্রতর', 'NNP'], ['এটা', 'QTC'], ['প্রধান', 'JJ'], ['গুন', 'NNP'], ['ছিল ', 'VAUX'], ['ওয়', 'PRP'], ['কখনো', 'PRQ'], ['সত্য ', 'NN'], ['কথা', 'NN'], ['কোতো ', 'VF'], ['না', 'NEG'], ['।', 'PUNC'], ['নাম', 'NN'], ['সত্যব্রত ', 'NNP'], ['হলেও', 'VAUX'], ['মিথ্যাকেই ', 'NN'], ['ওয়', 'PRP'], ['যেনো', 'RPD'], ['জীবনের', 'NN'], ['ব্রত', 'NN'], ['করিছিলো', 'VF'], ['।', 'PUNC'], ['এমন', 'DMD'], ['করি', 'VNF'], ['আর ', 'CCD'], ['কেউ', 'DMI'], ['মিথ্যা', 'NN'], ['কোবা', 'VNG'], ['পারে', 'VF'], ['কি', 'RPD'], ['না ', 'NEG'], ['সন্দেহ', 'NN'], ['।', 'PUNC'], ['ধরা', 'VNG'], ['পরি', 'VNF'], ['গেলেও', 'VNF'], ['এনাও', 'INTF'], ['ঘাবড়াত', 'VF'], ['না', 'NEG'], ['।', 'PUNC'], ['বিশ্বব্রক্ষ্মান্ডের', 'NN'], ['মত', 'PRC'], ['বড়লোক', 'NN'], ['সবাই', 'PRP'], ['অর', 'DMD'], ['কুটুম', 'NN'], ['।', 'PUNC'], ['হামার', 'PRP'], ['বাংলার', 'NN'], ['মাস্টার', 'NN'], ['হেমাঙ্গ', 'NNP'], ['বাবু', 'NNP'], ['ছিলেন', 'VAUX'], ['এনা', 'INTF'], ['কবি', 'NN'], ['প্রকৃতির', 'NN'], ['।', 'PUNC'], ['তিনি', 'PRP'], 

In [ ]:
train_tagged_words = []
with open("train.txt", mode = "r") as f:
  g = f.read()
  k = g.split('\n')
  for i in k:
    t = i.split('\t')
    if t[0] != '':
      train_tagged_words.append(t)
print(train_tagged_words)
print(len(train_tagged_words))


[['ভূতনাথ', 'NNP'], ['বাবুঘরের', 'NNP'], ['কাঠের', 'NN'], ['কারবারত', 'NN'], ['মেলা', 'QTF'], ['পয়সা', 'NN'], ['।', 'PUNC'], ['তার', 'PRP'], ['সাথে', 'PSP'], ['সুদেত', 'NN'], ['টাকা', 'NN'], ['ধার', 'NN'], ['দেওয়া', 'VNG'], ['তার', 'PRP'], ['আর', 'CCD'], ['এক', 'QTC'], ['ব্যাবসা', 'NN'], ['।', 'PUNC'], ['এই', 'DMD'], ['সব', 'QTF'], ['করি', 'VNF'], ['মেলা', 'QTF'], ['টাকা', 'NN'], ['জমাই', 'VNF'], ['ফেলিছে', 'VF'], ['।', 'PUNC'], ['কিন্তু', 'CCD'], ['তার', 'PRP'], ['মতো', 'PRC'], ['কিপটা', 'JJ'], ['লোক', 'NN'], ['দেখা', 'VNG'], ['যায়', 'VNF'], ['না', 'NEG'], ['।', 'PUNC'], ['চুনা', 'NNP'], ['মাছ', 'NN'], ['আর', 'CCD'], ['পুঁই', 'NNP'], ['চচ্চরি', 'NNP'], ['খায়', 'VF'], ['।', 'PUNC'], ['নাপিতক', 'NN'], ['পয়সা', 'NN'], ['দিবা', 'VNG'], ['লাগবি', 'VAUX'], ['ওই', 'DMD'], ['তনে', 'CCD'], ['চুল', 'NN'], ['দাড়ি', 'NN'], ['কাটাবা', 'VINF'], ['যায়', 'VNF'], ['না', 'NEG'], ['।', 'PUNC'], ['গাওত', 'NN'], ['এটা', 'QTC'], ['ওভার', 'NNP'], ['কোট', 'NNP'], ['পড়ি', 'VNF'], ['থাকেন', 'VF'], ['সা

In [ ]:
print("Length of Train sent and test set:")
print(len(train_tagged_words))
print(len(test_tagged_words))

Length of Train sent and test set:
1450
895


In [ ]:
# print some of the tagged words.
print(train_tagged_words[:10])
print(test_tagged_words[:10])

[['ভূতনাথ', 'NNP'], ['বাবুঘরের', 'NNP'], ['কাঠের', 'NN'], ['কারবারত', 'NN'], ['মেলা', 'QTF'], ['পয়সা', 'NN'], ['।', 'PUNC'], ['তার', 'PRP'], ['সাথে', 'PSP'], ['সুদেত', 'NN']]
[['সত্যব্রতর', 'NNP'], ['এটা', 'QTC'], ['প্রধান', 'JJ'], ['গুন', 'NNP'], ['ছিল ', 'VAUX'], ['ওয়', 'PRP'], ['কখনো', 'PRQ'], ['সত্য ', 'NN'], ['কথা', 'NN'], ['কোতো ', 'VF']]


In [ ]:
#use set datatype to check how many unique tags are present in training data
tags = []
for i in train_tagged_words:
  if i[1] not in tags:
    tags.append(i[1])
print("Total tags:")
print(len(tags))
print("All tags:")
print(tags)

Total tags:
39
All tags:
['NNP', 'NN', 'QTF', 'PUNC', 'PRP', 'PSP', 'VNG', 'CCD', 'QTC', 'DMD', 'VNF', 'VF', 'PRC', 'JJ', 'NEG', 'VAUX', 'VINF', 'QTO', 'NST', 'RB', 'ECH', 'SYM', 'INTF', 'DMQ', 'QTI', 'RPD', 'PRI', 'QRI', 'INJ', 'CCS', 'DMI', 'SYM ', 'PUNC ', 'CL', 'VM', 'PR', 'PRL', 'PRQ', 'RBQ']


In [ ]:
# check total words in vocabulary
vocab = []
for i in train_tagged_words:
  if i[0] not in vocab:
    vocab.append(i[0])
print("Total vocab:")
print(len(vocab))
print("All vocab:")
print(vocab)

Total vocab:
805
All vocab:
['ভূতনাথ', 'বাবুঘরের', 'কাঠের', 'কারবারত', 'মেলা', 'পয়সা', '।', 'তার', 'সাথে', 'সুদেত', 'টাকা', 'ধার', 'দেওয়া', 'আর', 'এক', 'ব্যাবসা', 'এই', 'সব', 'করি', 'জমাই', 'ফেলিছে', 'কিন্তু', 'মতো', 'কিপটা', 'লোক', 'দেখা', 'যায়', 'না', 'চুনা', 'মাছ', 'পুঁই', 'চচ্চরি', 'খায়', 'নাপিতক', 'দিবা', 'লাগবি', 'ওই', 'তনে', 'চুল', 'দাড়ি', 'কাটাবা', 'গাওত', 'এটা', 'ওভার', 'কোট', 'পড়ি', 'থাকেন', 'সাড়া', 'বচ্ছর', 'লাভের', 'প্রথম', 'বারের', 'দিয়ে', 'চোরাবাজারত', 'থেকি', 'দু', 'তিন', 'হাত', 'ফেরতা', 'কোটটা', 'কিনিছিলেন', 'সস্তা', 'দামে', 'সে', 'আজ', 'বত্রিশ', 'হলো', 'আছেন', 'ওজনে', 'সের', 'পাঁচেক', 'হবি', 'মিলি', 'কম', 'পঁচিশটা', 'পকট', 'বাবু', 'রাস্তায়', 'ধুলাবালি', 'সানতে', 'রাস্তা', 'পার', 'হয়ে', 'অবশেষে', 'মোর', 'গোরত', 'আসি', 'হাজির', 'হলেন', 'মোক', 'দেখেই', 'ভেউ', 'কান্দি', 'কোলো', ',', '"', 'সব্বনাশ', 'হয়ি', 'গেল', 'হিরু', 'এক্কেরে', 'পথে', 'বসাইছে', 'যায়ে', 'কনু', 'কি', 'হইছে', 'কন', 'পাহাড়াওয়ালা', 'ডাকিম', '?', 'পাহারাওয়ালার', 'নাম', 'কানত', 'যাওয়ায়', 'বুড়

Define Emission Probability and Transition Probability

In [ ]:
# compute Emission Probability
def word_given_tag(word, tag, train_bag = train_tagged_words):
    tag_list = [pair for pair in train_bag if pair[1]==tag]
    count_tag = len(tag_list)#total number of times the passed tag occurred in train_bag
    w_given_tag_list = [pair[0] for pair in tag_list if pair[0]==word]
#now calculate the total number of times the passed word occurred as the passed tag.
    count_w_given_tag = len(w_given_tag_list)
    return (count_w_given_tag, count_tag)
# compute  Transition Probability
def t2_given_t1(t2, t1, train_bag = train_tagged_words):
    tags = [pair[1] for pair in train_bag]
    count_t1 = len([t for t in tags if t==t1])
    count_t2_t1 = 0
    for index in range(len(tags)-1):
        if tags[index]==t1 and tags[index+1] == t2:
            count_t2_t1 += 1
    return (count_t2_t1, count_t1)

In [ ]:
# creating t x t transition matrix of tags, t= no of tags
# Matrix(i, j) represents P(jth tag after the ith tag)
tags_matrix = np.zeros((len(tags), len(tags)), dtype='float32')
for i, t1 in enumerate(list(tags)):
    for j, t2 in enumerate(list(tags)):
        tags_matrix[i, j] = t2_given_t1(t2, t1)[0]/t2_given_t1(t2, t1)[1]

In [ ]:
print(tags_matrix)

[[0.2753623  0.22463769 0.01449275 ... 0.00724638 0.         0.        ]
 [0.05236908 0.3042394  0.02493766 ... 0.00249377 0.00249377 0.00249377]
 [0.         0.51724136 0.10344828 ... 0.         0.         0.        ]
 ...
 [0.         0.         0.         ... 0.         0.         0.        ]
 [0.         0.25       0.         ... 0.         0.         0.        ]
 [0.         0.         0.         ... 0.         0.         0.        ]]


In [ ]:
# convert the matrix to a dataframe for better readability
#the table is same as the transition table shown in section 3 of article
tags_df = pd.DataFrame(tags_matrix, columns = list(tags), index=list(tags))
print(tags_df)

            NNP        NN       QTF      PUNC       PRP       PSP       VNG  \
NNP    0.275362  0.224638  0.014493  0.224638  0.000000  0.036232  0.000000   
NN     0.052369  0.304239  0.024938  0.134663  0.017456  0.037406  0.027431   
QTF    0.000000  0.517241  0.103448  0.068966  0.000000  0.000000  0.034483   
PUNC   0.151042  0.244792  0.005208  0.000000  0.062500  0.000000  0.000000   
PRP    0.023256  0.395349  0.000000  0.046512  0.000000  0.116279  0.000000   
PSP    0.033333  0.333333  0.033333  0.200000  0.000000  0.000000  0.000000   
VNG    0.000000  0.307692  0.000000  0.076923  0.076923  0.000000  0.000000   
CCD    0.193548  0.419355  0.032258  0.000000  0.064516  0.000000  0.000000   
QTC    0.028571  0.628571  0.000000  0.028571  0.000000  0.000000  0.000000   
DMD    0.083333  0.375000  0.125000  0.000000  0.000000  0.041667  0.000000   
VNF    0.085106  0.170213  0.063830  0.042553  0.021277  0.000000  0.000000   
VF     0.083969  0.083969  0.030534  0.480916  0.022

Define Viterbi Algorithm

In [ ]:
def Viterbi(words, train_bag = train_tagged_words):
    state = []
    T = list(set([pair[1] for pair in train_bag]))

    for key, word in enumerate(words):
        #initialise list of probability column for a given observation
        p = []
        for tag in T:
            if key == 0:
                transition_p = 0
            else:
                transition_p = tags_df.loc[state[-1], tag]

            # compute emission and state probabilities
            emission_p = word_given_tag(words[key], tag)[0]/word_given_tag(words[key], tag)[1]
            state_probability = emission_p * transition_p
            p.append(state_probability)

        pmax = max(p)
        # getting state for which probability is maximum
        state_max = T[p.index(pmax)]
        state.append(state_max)
    return list(zip(words, state))

Create Test set

In [ ]:
test_run = test_tagged_words
print(len(test_run))
# list of tagged words
test_seq = [tuple(i) for i in test_tagged_words]
print("Test sentences:")
print(test_seq)
# list of untagged words
test_words = [tup[0] for tup in test_seq]

895
Test sentences:
[('সত্যব্রতর', 'NNP'), ('এটা', 'QTC'), ('প্রধান', 'JJ'), ('গুন', 'NNP'), ('ছিল ', 'VAUX'), ('ওয়', 'PRP'), ('কখনো', 'PRQ'), ('সত্য ', 'NN'), ('কথা', 'NN'), ('কোতো ', 'VF'), ('না', 'NEG'), ('।', 'PUNC'), ('নাম', 'NN'), ('সত্যব্রত ', 'NNP'), ('হলেও', 'VAUX'), ('মিথ্যাকেই ', 'NN'), ('ওয়', 'PRP'), ('যেনো', 'RPD'), ('জীবনের', 'NN'), ('ব্রত', 'NN'), ('করিছিলো', 'VF'), ('।', 'PUNC'), ('এমন', 'DMD'), ('করি', 'VNF'), ('আর ', 'CCD'), ('কেউ', 'DMI'), ('মিথ্যা', 'NN'), ('কোবা', 'VNG'), ('পারে', 'VF'), ('কি', 'RPD'), ('না ', 'NEG'), ('সন্দেহ', 'NN'), ('।', 'PUNC'), ('ধরা', 'VNG'), ('পরি', 'VNF'), ('গেলেও', 'VNF'), ('এনাও', 'INTF'), ('ঘাবড়াত', 'VF'), ('না', 'NEG'), ('।', 'PUNC'), ('বিশ্বব্রক্ষ্মান্ডের', 'NN'), ('মত', 'PRC'), ('বড়লোক', 'NN'), ('সবাই', 'PRP'), ('অর', 'DMD'), ('কুটুম', 'NN'), ('।', 'PUNC'), ('হামার', 'PRP'), ('বাংলার', 'NN'), ('মাস্টার', 'NN'), ('হেমাঙ্গ', 'NNP'), ('বাবু', 'NNP'), ('ছিলেন', 'VAUX'), ('এনা', 'INTF'), ('কবি', 'NN'), ('প্রকৃতির', 'NN'), ('।', 'PUNC'

Predict the tags for test set using Viterbi algorithm

In [ ]:
#Test 10 sentences to check the accuracy as testing the whole training set takes huge amount of time
start = time.time()
tagged_seq = Viterbi(test_words)
end = time.time()
difference = end-start
print("Predicted sequence:")
print(tagged_seq)
print("Predicted sequence lengh:")
print(len(tagged_seq) )
print(tagged_seq[-10:])
print("Time taken in seconds: ", difference)

Predicted sequence:
[('সত্যব্রতর', 'DMI'), ('এটা', 'DMI'), ('প্রধান', 'DMI'), ('গুন', 'DMI'), ('ছিল ', 'DMI'), ('ওয়', 'DMI'), ('কখনো', 'DMI'), ('সত্য ', 'DMI'), ('কথা', 'DMI'), ('কোতো ', 'DMI'), ('না', 'DMI'), ('।', 'DMI'), ('নাম', 'NN'), ('সত্যব্রত ', 'DMI'), ('হলেও', 'DMI'), ('মিথ্যাকেই ', 'DMI'), ('ওয়', 'DMI'), ('যেনো', 'DMI'), ('জীবনের', 'DMI'), ('ব্রত', 'DMI'), ('করিছিলো', 'DMI'), ('।', 'DMI'), ('এমন', 'DMI'), ('করি', 'DMI'), ('আর ', 'DMI'), ('কেউ', 'DMI'), ('মিথ্যা', 'DMI'), ('কোবা', 'DMI'), ('পারে', 'DMI'), ('কি', 'DMI'), ('না ', 'DMI'), ('সন্দেহ', 'DMI'), ('।', 'DMI'), ('ধরা', 'DMI'), ('পরি', 'DMI'), ('গেলেও', 'DMI'), ('এনাও', 'DMI'), ('ঘাবড়াত', 'DMI'), ('না', 'DMI'), ('।', 'DMI'), ('বিশ্বব্রক্ষ্মান্ডের', 'DMI'), ('মত', 'DMI'), ('বড়লোক', 'DMI'), ('সবাই', 'DMI'), ('অর', 'DMI'), ('কুটুম', 'DMI'), ('।', 'DMI'), ('হামার', 'PRP'), ('বাংলার', 'NNP'), ('মাস্টার', 'DMI'), ('হেমাঙ্গ', 'DMI'), ('বাবু', 'DMI'), ('ছিলেন', 'DMI'), ('এনা', 'DMI'), ('কবি', 'DMI'), ('প্রকৃতির', 'NN'), ('।'

Calculate Accuracy

In [ ]:
# accuracy
check = [i for i, j in zip(tagged_seq, test_seq) if i == j]
print(check)
accuracy = len(check)/len(tagged_seq)
print('Viterbi Algorithm Accuracy: ',accuracy*100)

[('নাম', 'NN'), ('কেউ', 'DMI'), ('হামার', 'PRP'), ('প্রকৃতির', 'NN'), ('।', 'PUNC'), ('তিনি', 'PRP'), ('মোর', 'PRP'), ('মুই', 'PRP'), ('কেউ', 'DMI'), ('কেউই', 'DMI'), ('তোমার', 'PRP'), ('তার', 'PRP'), ('তার', 'PRP'), ('নাম', 'NN'), ('?', 'PUNC'), ('"', 'SYM'), ('সত্যব্রত', 'NNP'), ('তিনি', 'PRP'), ('মুই', 'PRP'), ('তো', 'RPD'), ('তোমার', 'PRP'), ('কদিন', 'DMI'), ('কদিন', 'DMI'), ('তোমার', 'PRP'), (',', 'PUNC'), ('বাড়িত', 'NN'), ('মোর', 'PRP'), ('মোর', 'PRP'), ('কোথাকার', 'DMI'), ('তোমার', 'PRP'), ('নাম', 'NN'), ('সত্যব্রত', 'NNP'), ('তোমার', 'PRP'), ('তোমার', 'PRP'), ('বাড়িত', 'NN'), ('থাকেন', 'VF'), ('।', 'PUNC'), ('"', 'SYM'), ('হামার', 'PRP'), ('খুব', 'INTF'), ('কেউ ', 'DMI'), ('তো', 'RPD'), ('পথের', 'NN'), ('কথায়', 'NN'), ('মনে', 'NN'), ('হাতে', 'NN'), ('রাস্তা', 'NN'), ('পেরিয়ে', 'VNF'), ('আজ', 'NN'), ('ঘন্টার', 'NN'), ('আজ', 'NN'), ('আর', 'CCD'), ('মুখে', 'NN'), ('মনে', 'NN'), ('পথে', 'NN'), (',', 'PUNC'), ('শুরু', 'NN'), ('হতে', 'VM'), ('পারে', 'VAUX'), ('প্রথমেই', 'NN'), ('শে

In [ ]:
res = [(i[0], i[1], j[1]) for i, j in zip(tagged_seq, test_seq) if i[0] == j[0]]
with open("res.txt", "w") as f:
  for i in res:
    string = "\t".join(i)
    f.write(string+"\n")